In [412]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 
from typing import Any, Optional, List

import qubx
%qubxd dev 

%load_ext autoreload
%autoreload 2

from qubx import lookup, logger
from qubx.data.readers import CsvStorageDataReader, MultiQdbConnector, QuestDBConnector, DataReader, AsTimestampedRecords, AsQuotes, RestoreTicksFromOHLC, AsPandasFrame, RestoreTicksFromOHLC, AsOhlcvSeries
from qubx.pandaz.utils import *
from qubx.data.helpers import  InMemoryCachedReader

 >  [dev] installing cython rebuilding hook
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Aux Data interface

In [20]:
r_csv = CsvStorageDataReader("../tests/data/csv")
r_mqc = MultiQdbConnector("xlydian-data")
r_qdb = QuestDBConnector(host="xlydian-data")

In [10]:
r_csv.get_aux_data_ids(), r_qdb.get_aux_data_ids(), r_mqc.get_aux_data_ids()

(set(),
 {'average_quote_volume', 'candles', 'fundamental_data', 'symbols'},
 {'average_quote_volume', 'candles', 'fundamental_data', 'symbols'})

In [ ]:
r_mqc.get_aux_data('candles', exchange='BINANCE.UM', 
                   symbols=['BTCUSDT', 'ETHUSDT'], 
                   start='2024-01-01', stop='2024-01-31')

In [26]:
r_qdb.get_aux_data('fundamental_data', exchange='BINANCE.UM', 
                #    symbols=['BTCUSDT', 'ETHUSDT'], 
                   start='2024-01-01', stop='2024-02-01')

metric                  market_cap       price        volume    volume_usd
timestamp  symbol                                                         
2024-01-01 1INCHUSDT  4.810212e+08     0.42810  2.528007e+07  1.082240e+07
           AAVEUSDT   1.604350e+09   108.78000  1.155772e+06  1.257249e+08
           ACHUSDT    1.051147e+08     0.02136  4.327358e+08  9.243236e+06
           ADAUSDT    2.076801e+10     0.59410  3.323489e+08  1.974485e+08
           AGIXUSDT   3.989788e+08     0.31970  4.189333e+07  1.339330e+07
...                            ...         ...           ...           ...
2024-01-31 YFIUSDT    2.435439e+08  7309.00000  1.259769e+03  9.207652e+06
           ZECUSDT    3.307871e+08    22.59000  5.673166e+05  1.281568e+07
           ZENUSDT    1.160340e+08     7.94800  5.515012e+05  4.383332e+06
           ZILUSDT    3.599965e+08     0.02046  3.587373e+08  7.339764e+06
           ZRXUSDT    2.798377e+08     0.33150  1.322362e+08  4.383631e+07

[5456 rows x 4 columns]

# Handy reader

In [433]:
F = '4h'
_P = lambda ds: [print(f"{s} : {len(v)} | {v.index[0]} : {v.index[-1]}") for s, v in ds.items()]
mcr = InMemoryCachedReader('BINANCE.UM', r_mqc, F)

In [ ]:
sorted(r_mqc.get_symbols("BINANCE.UM"))

In [ ]:
mcr["BTCUSDT", '2020-01-01':'2024-01-01']

In [453]:
mcr.read("BTCUSDT", "2020-01-01", "2024-08-10", transform=AsPandasFrame())

,close,count,high,low,open,quote_volume,taker_buy_quote_volume,taker_buy_volume,volume
timestamp,,,,,,,,,
2020-01-01 00:00:00,7221.65,19586,7239.74,7170.15,7189.43,8.690107e+07,4.582436e+07,6356.286,12055.759
2020-01-01 04:00:00,7205.26,10298,7230.00,7193.52,7221.80,4.141070e+07,2.060753e+07,2855.461,5738.351
2020-01-01 08:00:00,7192.65,23427,7234.57,7174.00,7205.26,7.223947e+07,3.472622e+07,4823.628,10036.291
2020-01-01 12:00:00,7229.49,19184,7260.43,7191.79,7192.74,7.951657e+07,4.301933e+07,5956.599,11010.526
2020-01-01 16:00:00,7226.35,12655,7248.00,7210.81,7229.66,5.631450e+07,2.967875e+07,4104.195,7787.863
...,...,...,...,...,...,...,...,...,...
2024-08-09 04:00:00,60844.00,442202,61361.90,60555.00,61327.00,2.002858e+09,9.907642e+08,16258.208,32873.136
2024-08-09 08:00:00,60633.50,582821,61184.90,60150.00,60844.10,2.692431e+09,1.298362e+09,21409.736,44404.396
2024-08-09 12:00:00,59786.40,1178944,61089.70,59500.00,60633.50,5.765730e+09,2.804591e+09,46535.184,95689.816


In [461]:
mcr["ATAUSDT"]

,open,high,low,close,volume,quote_volume,count,taker_buy_volume,taker_buy_quote_volume
timestamp,,,,,,,,,
2021-08-31 00:00:00,1.5405,1.6809,1.4930,1.5738,6198496.0,9.954537e+06,38280,2497084.0,4014575.000
2021-08-31 04:00:00,1.5739,1.6249,1.4611,1.5171,7461774.0,1.145464e+07,47185,3004177.0,4617676.500
2021-08-31 08:00:00,1.5171,1.5249,1.4701,1.5019,4051722.0,6.051100e+06,28652,1805285.0,2696990.800
2021-08-31 12:00:00,1.5014,1.5060,1.4509,1.4578,4406265.0,6.509029e+06,31365,1964782.0,2904614.500
2021-08-31 16:00:00,1.4578,1.4617,1.4010,1.4113,3900691.0,5.555895e+06,23925,1672413.0,2382999.800
...,...,...,...,...,...,...,...,...,...
2024-08-09 04:00:00,0.0754,0.0756,0.0740,0.0751,5285948.0,3.957739e+05,3550,2313045.0,173241.808
2024-08-09 08:00:00,0.0751,0.0758,0.0742,0.0754,4872003.0,3.658971e+05,3576,2415818.0,181443.264
2024-08-09 12:00:00,0.0754,0.0758,0.0735,0.0739,7771951.0,5.801331e+05,5964,3758468.0,280545.856


In [356]:
_P(mcr._get_smbs_at(['LTCUSDT', 'BTCUSDT'], "2023-01-01", "2024-01-01" ));

LTCUSDT : 2190 | 2023-01-01 00:00:00 : 2023-12-31 20:00:00
BTCUSDT : 2190 | 2023-01-01 00:00:00 : 2023-12-31 20:00:00


In [357]:
_P(mcr._get_smbs_at(['LTCUSDT', 'BTCUSDT', 'ETHUSDT', 'SOLUSDT', 'XRPUSDT'], "2021-01-01", "2024-06-01" ));

LTCUSDT : 7452 | 2021-01-01 00:00:00 : 2024-05-31 20:00:00
BTCUSDT : 7458 | 2021-01-01 00:00:00 : 2024-05-31 20:00:00
ETHUSDT : 7482 | 2021-01-01 00:00:00 : 2024-05-31 20:00:00
SOLUSDT : 7452 | 2021-01-01 00:00:00 : 2024-05-31 20:00:00
XRPUSDT : 7452 | 2021-01-01 00:00:00 : 2024-05-31 20:00:00


In [358]:
mcr1 = InMemoryCachedReader('BINANCE.UM', r_mqc, F)
_P(x1:=mcr1._get_smbs_at(['LTCUSDT'], "2021-01-01", "2024-06-01"))
_P(x2:=mcr._get_smbs_at(['LTCUSDT'],  "2021-01-01", "2024-06-01"))
all(x1['LTCUSDT']==x2['LTCUSDT'])

LTCUSDT : 7452 | 2021-01-01 00:00:00 : 2024-05-31 20:00:00
LTCUSDT : 7452 | 2021-01-01 00:00:00 : 2024-05-31 20:00:00


True

In [422]:
class X:
    def __getitem__(self, keys):
        _start, _stop = None, None
        _instruments = []
        _as_dict = False
        if isinstance(keys, (tuple)):
            for k in keys:
                if isinstance(k, slice):
                    _start, _stop = k.start, k.stop
                if isinstance(k, (list, tuple)):
                    _instruments = k
                    _as_dict = True
                if isinstance(k, str):
                    _instruments.append(k)
        else:
            if isinstance(keys, (list, tuple)):
                _instruments.extend(keys)
                _as_dict = True
            else:
                _instruments.append(keys)
        _as_dict |= len(_instruments) > 1
        print(f" >>> Get data for {_instruments} from {_start} to {_stop} | {_as_dict}")

s = X()
s['2023-01-01':'2024-01-01' ]
s[["BTCUSDT"], '2023-01-01':'2024-01-01' ]
s["BTCUSDT", '2023-01-01':'2024-01-01' ]
s[['BTC']]
s['BTC', 'ETH']
# s[{'b': 1}]

 >>> Get data for [slice('2023-01-01', '2024-01-01', None)] from None to None | False
 >>> Get data for ['BTCUSDT'] from 2023-01-01 to 2024-01-01 | True
 >>> Get data for ['BTCUSDT'] from 2023-01-01 to 2024-01-01 | False
 >>> Get data for ['BTC'] from None to None | True
 >>> Get data for ['BTC', 'ETH'] from None to None | True
